In [5]:
# Install needed packages
# biopython for biological analitical methods
# elementpath for tree itiration
# pandas for the construction of a dataframe from the xml file
# SQLAlchemy and db-sqlite3 for database parsing from the dataframe
!pip install biopython
!pip install elementpath
!pip install pandas
!pip install SQLAlchemy
!pip install db-sqlite3
!pip install pandasql
!pip3 install taxoniq
!pip install nglview
!pip install biopandas
!pip install tabulate
!pip install collection
!pip install contextlib2
!pip install fuzzywuzzy
!pip install pyhmmer
!pip install sto
!pip install bio
!pip install hmmlearn
!pip install unipressed
!pip install uniprot
!pip install requests
!pip install subprocess
!pip install dict2xml 
!pip install minidom-ext
!pip install flatten-json
!pip install cdifflib



Defaulting to user installation because normal site-packages is not writeable
  Using cached sdRDM-0.0.3-py3-none-any.whl (47 kB)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import sys
!{sys.executable} -m pip install git+https://github.com/JR-1991/software-driven-rdm.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/JR-1991/software-driven-rdm.git to /tmp/pip-req-build-gy9wlp2s
  Running command git clone --filter=blob:none --quiet https://github.com/JR-1991/software-driven-rdm.git /tmp/pip-req-build-gy9wlp2s
  Resolved https://github.com/JR-1991/software-driven-rdm.git to commit 6d29fddb754c2dfce5f3ddec709bcf84a18e29b3
  Preparing metadata (setup.py) ... done


In [35]:
# query protein sequence input
from Bio import SeqIO
protein_seq = SeqIO.read("sequence.842.fasta", "fasta")

In [9]:
# double check the sequence
protein_seq.seq

Seq('HPETLVKVKDAEDQLGARVGYIELDLNSGKILESFRPEERFPMMSTFKVLLCGA...KHW')

In [10]:
# run an online blast search in the NCBI database
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastp", "pdb", protein_seq.seq) # pdb - protein data bank

In [11]:
# save the ncbi search results as an xml document
with open("TEM-1.xml", "w") as save_to:
    save_to.write(result_handle.read())
    result_handle.close()

In [2]:
# import sdRDM package
import sdRDM
from sdRDM import DataModel
from sdRDM.database import build_sql_database

In [3]:
#import the data base
pyeed = DataModel.from_git("https://github.com/maxim945/PyEED-Fullprototype.git")
#visualize for each separate table
pyeed.ProteinSequence.visualize_tree()
#pyeed.DNASequence.visualize_tree()
#pyeed.StandardNumberingScheme.visualize_tree()
#pyeed.ProteinDatabase.visualize_tree()

ProteinSequence
├── id
├── protein_sequence_id
├── name
├── amino_acid_sequence
├── protein_database_name
│   └── ProteinDatabase
│       ├── id
│       ├── database
│       └── link_to_database
└── protein_organism_id
    └── Organism
        ├── id
        ├── organism_name
        └── ncbi_taxonomy_id


This part of the code updated to itirate the data directly into the database, unlike in the previouse versions where you had to make a dataframe and then send it using SQL into the database here it goes directly from the source.
NOTE!, to the full pipline like in the previouse versions you still should turn it into a dataframe for the 

In [8]:
# convert the xml to a dataframe
from sdRDM import DataModel
import xml.etree.ElementTree as ET

# PARSE XML
tree = ET.parse('TEM-1.xml')
root = tree.getroot()

# Create a dataframe with 50 rows and 1 column
# search for the specific hits in the xml file
for child in root.iter():
    if child.tag == 'BlastOutput_db':
        BlastOutput_db=child.text.strip()
    elif child.tag == 'Hit_accession':
        Hit_accession=child.text.strip()
    elif child.tag == 'Hit_def':
        Hit_def=child.text.strip()
    elif child.tag == 'Hsp_midline':
        Hsp_midline=child.text.strip()

        #export the data directly into the database
        #chose the table (ProteinSequence) to which the data will be exported
        #to each [Column Name] = [XML data] 
        #alocate into the database
        dataset = pyeed.ProteinSequence(
        protein_sequence_id=Hit_accession,
        name=Hit_def,
        amino_acid_sequence=Hsp_midline)
        dataset.to_sql(loc="./TEM-1.db")

In [4]:
#build the database for each table
build_sql_database(pyeed.ProteinSequence, loc="./TEM-1.db")
#build_sql_database(pyeed.ProteinDatabase, loc="./TEM-1.db")
#build_sql_database(pyeed.StandardNumberingScheme, loc="./TEM-1.db")